## Proyecto Datos estadisticos de Bancos BCParaguay; 
Data Engineer:  Rodrigo Figueredo 
Objetivo: se necesita un proceso consistente de extraccion de datos para cargarlo en un Datawarehouse en SQL, el proceso deberá correr de forma mensual y que contenga los datos consistente y evitar a manipulación manual del excel origen.

Antes que nada debemos monitorear, depurar y auditar cada paso. 


In [4]:

# Configuración logging
# Primero, configuramos el logger al inicio del script:
import os
import logging
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import logging
import pyodbc

# Obtener la ruta actual
ruta_actual = os.getcwd()
# Configurar logging
logging.basicConfig(
    filename=f'{ruta_actual}/etl_proceso.log',       # Archivo de log
    level=logging.INFO,               # Nivel de log: DEBUG, INFO, WARNING, ERROR, CRITICAL
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='w'                      # 'w' sobrescribe cada vez, 'a' acumula
)

logger = logging.getLogger()
print('Se inicializa parametros')

Se inicializa parametros


🧩1. Extracción (Extract)

In [8]:
import pandas as pd

## ruta del archivo
excel_file = 'C:/Users/rfigu/Documents/Python Scripts/BoletinBCP/Tablas_Boletin_Bancos_Jul25.xlsx'

def extraer_excel(excel_file):
    try:
        logger.info(f"Iniciando extracción desde: {excel_file}")
        sheets_dict = pd.read_excel(excel_file, sheet_name=None)
        # Acceder a cada hoja como DataFrame
        for sheet_name, df in sheets_dict.items():
            print(f"Hoja: {sheet_name}, Filas: {len(df)}")
        # logger.info(f"Iniciando extracción desde: {excel_file}")
        # sheets_dict = pd.read_excel(excel_file, sheet_name=None)
        # # Acceder a cada hoja como DataFrame
        # for sheet_name, df in sheets_dict.items():
        #     print(f"Hoja: {sheet_name}, Filas: {len(df)}")
        #     logger.info(f"Hoja: {sheet_name}, Filas: {len(df)}")
        return sheets_dict
    except Exception as e:
        logger.error(f"Error en la extracción: {e}")
        raise

In [ ]:
# Cargar todas las hojas del Excel en un diccionario
excel_file = 'C:/Users/rfigu/Documents/Python Scripts/BoletinBCP/Tablas_Boletin_Bancos_Jul25.xlsx'
sheets_dict = pd.read_excel(excel_file, sheet_name=None)

# Acceder a cada hoja como DataFrame
for sheet_name, df in sheets_dict.items():
    print(f"Hoja: {sheet_name}, Filas: {len(df)}")

In [ ]:
# Obtener ruta actual
ruta_actual = os.getcwd()
logger.info(f"Ruta actual de ejecución: {ruta_actual}")

# Ruta del archivo Excel
excel_file = 'C:/Users/rfigu/Documents/Python Scripts/BoletinBCP/Tablas_Boletin_Bancos_Jul25.xlsx'

# Extracción de hojas
try:
    logger.info(f"Iniciando lectura del archivo Excel: {excel_file}")
    sheets_dict = pd.read_excel(excel_file, sheet_name=None)
    logger.info(f"Lectura exitosa. Hojas encontradas: {list(sheets_dict.keys())}")
    
    # Acceder a cada hoja como DataFrame
    for sheet_name, df in sheets_dict.items():
        logger.info(f"Hoja: {sheet_name}, Filas: {len(df)}")
        print(f"Hoja: {sheet_name}, Filas: {len(df)}")

except Exception as e:
    logger.error(f"Error al leer el archivo Excel: {e}")
    print(f"Error: {e}")

Hoja: EEFF, Filas: 224012
Hoja: Ratios, Filas: 82673
Hoja: Carteras, Filas: 49860
Hoja: Credito Sector, Filas: 43629
Hoja: CreditoActividad, Filas: 9333
Hoja: Categoría_Creditos, Filas: 14500
Hoja: TC, Filas: 3872
Hoja: Canales_Person, Filas: 13198
Hoja: Inhab, Filas: 7443


1.1 EDA - Exploratory Data Analysis

In [15]:
## Vista previa de los datos la hoja
sheets_dict.get('Carteras').head()

,Fecha,Codigo Entidad,Codigo Cuenta,Codigo Moneda,Importe
0,2016-01-31,1002,Cartera Vigente,6200,2.383314e+05
1,2016-01-31,1002,Cartera Vigente,6900,2.075860e+06
2,2016-01-31,1002,Cartera Vigente Neta de Previsiones,6200,2.383314e+05
3,2016-01-31,1002,Cartera Vigente Neta de Previsiones,6900,2.051322e+06
4,2016-01-31,1002,Renovados,6200,1.601913e+04


In [16]:
## Vista previa de los tipos de datos la hoja
sheets_dict.get('TC').dtypes

Fecha             datetime64[ns]
Codigo Entidad             int64
Clasificación             object
Total                    float64
dtype: object

🔄 2. Transformación (Transform)

Aquí puedes aplicar limpieza, normalización, validación, etc. Ejemplo:


In [19]:
# Función de transformación con logging
def transforBol(df, nombre_hoja):
    try:
        logger.info(f"Iniciando transformación de hoja: {nombre_hoja}")

        # Eliminar filas vacías
        df = df.dropna(how='all')
        logger.info(f"Filas vacías eliminadas en hoja: {nombre_hoja}")

        # Renombrar columnas
        columnas_originales = df.columns.tolist()
        df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
        logger.info(f"Columnas renombradas en hoja: {nombre_hoja} — Originales: {columnas_originales} → Nuevas: {df.columns.tolist()}")

        # Convertir fechas
        if 'fecha' in df.columns:
            df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
            errores_fecha = df['fecha'].isna().sum()
            logger.info(f"Columna 'fecha' convertida en hoja: {nombre_hoja}. Errores de conversión: {errores_fecha}")
        else:
            logger.warning(f"No se encontró columna 'fecha' en hoja: {nombre_hoja}")

        logger.info(f"Transformación completada para hoja: {nombre_hoja}")
        return df

    except Exception as e:
        logger.error(f"Error al transformar hoja {nombre_hoja}: {e}")
        raise

# Aplicar transformación a cada hoja
datos_transformados = {}

for sheet, df in sheets_dict.items():
    datos_transformados[sheet] = transforBol(df, sheet)

In [ ]:
## version  sin Looger
def transforBol(df):
    # Eliminar filas vacías
    df = df.dropna(how='all')
    
    # Renombrar columnas
    df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
    
    # Convertir fechas
    if 'fecha' in df.columns:
        df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
    
    return df

# Aplicar transformación a cada hoja
datos_transformados = {sheet: transforBol(df) for sheet, df in sheets_dict.items()}

In [20]:
datos_transformados.get('Credito Sector').head(3)

,fecha,codigo_entidad,codigo_moneda,actividad_destino_vs2,cartera_vencida,cartera_vigente
0,2016-01-31,1002,6200,AGRICULTURA,2416.278988,152923.648836
1,2016-01-31,1002,6200,SECTOR FINANCIERO,0.000000,83374.169137
2,2016-01-31,1002,6200,VIVIENDA,0.000000,2033.573859


📥 3. Carga (Load)

In [ ]:
# Consulta de prueba
# query = "SELECT * FROM dbo.FortalezaTemp"
# df_query = pd.read_sql(query, connection)
# df_query.tail()

In [ ]:
# Parámetros de conexión
server = 'ASUSTUF\SQL22'  # Doble barra para escapar correctamente
database = 'BolBcp'

try:
    logger.info(f"Intentando conectar a SQL Server: {server}, Base de datos: {database}")
    
    connP = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        'Trusted_Connection=yes;'
    )
    
    logger.info("Conexión a SQL Server establecida exitosamente.")
    print('Conexión exitosa')

except pyodbc.Error as e:
    logger.error(f"Error al conectar a la base de datos: {str(e)}")
    print(f'Error al conectar a la base de datos: {str(e)}')


Error al conectar a la base de datos: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [67].  (67) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (67)')


In [ ]:
# query de ejemplo
# query = "SELECT * FROM dbo.FortalezaTemp"
# df_query = pd.read_sql(query, connection)
# df_query.tail()

In [46]:
for nombre_hoja, df in datos_transformados.items():
    # df ya es un DataFrame
    print(df)

            fecha  codigo_entidad                                sub_rubro  \
0      2016-01-31            1002     Administración de Cuentas Corrientes   
1      2016-01-31            1002  Bienes Realizados y adjudicados en pago   
2      2016-01-31            1002                            Bienes de Uso   
3      2016-01-31            1002                            Caja y Bancos   
4      2016-01-31            1002                            Caja y Bancos   
...           ...             ...                                      ...   
224007 2025-07-31            1047             RESULTADO ANTES DE PREVISIÓN   
224008 2025-07-31            1047              UTILIDAD ANTES DE IMPUESTOS   
224009 2025-07-31            1047              UTILIDAD ANTES DE IMPUESTOS   
224010 2025-07-31            1047                    UTILIDAD A DISTRIBUIR   
224011 2025-07-31            1047                    UTILIDAD A DISTRIBUIR   

        codigo_moneda        importe reporte  
0               

In [8]:
# Parámetros de conexión
server = 'ASUSTUF\SQL22'
database = 'BolBcp'
driver = 'ODBC Driver 17 for SQL Server'

# Crear cadena de conexión para SQLAlchemy
conexion_str = f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes"

try:
    engine = create_engine(conexion_str)
    print(f"✅ conexion exitosa")
    logger.info(f"Conexión a SQL Server establecida: {server}, Base de datos: {database}")
except Exception as e:
    print(f"❌ fallo en la conexion")
    logger.error(f"Error al crear engine SQLAlchemy: {e}")
    raise

✅ conexion exitosa


In [ ]:
# Cargar cada hoja transformada como tabla
for nombre_tabla, df in datos_transformados.items():
    try:
        logger.info(f"Iniciando carga de hoja: {nombre_tabla}")

        # Eliminar filas completamente vacías
        df = df.dropna(how='all')
        # Agregar columna de fecha y hora de carga
        df['fecha_carga'] = datetime.now()

        # Insertar usando pandas
        df.to_sql(name=nombre_tabla, con=engine, if_exists='replace', index=False)

        logger.info(f"Hoja '{nombre_tabla}' cargada exitosamente con {len(df)} filas.")
        print(f"✅ Hoja '{nombre_tabla}' cargada exitosamente con {len(df)} filas.")

    except Exception as e:
        logger.error(f"Error al cargar hoja '{nombre_tabla}': {e}")
        print(f"❌ Error al cargar hoja '{nombre_tabla}': {e}")

✅ Hoja 'EEFF' cargada exitosamente con 224012 filas.
✅ Hoja 'Ratios' cargada exitosamente con 82673 filas.
✅ Hoja 'Carteras' cargada exitosamente con 49860 filas.
✅ Hoja 'Credito Sector' cargada exitosamente con 43629 filas.
✅ Hoja 'CreditoActividad' cargada exitosamente con 9333 filas.
✅ Hoja 'Categoría_Creditos' cargada exitosamente con 14500 filas.
✅ Hoja 'TC' cargada exitosamente con 3872 filas.
✅ Hoja 'Canales_Person' cargada exitosamente con 13198 filas.
✅ Hoja 'Inhab' cargada exitosamente con 7443 filas.


Leer por nombre de tabla (si el Excel tiene tablas definidas)


In [6]:
## conexion a la Base datos SQL Server v2022
path = 'C:/Users/rfigu/Documents/Python Scripts/BoletinBCP/Dimensiones y referencias.xlsx'


dfDepara = pd.read_excel(path, sheet_name="entidad")
logger.info(f"Archivo cargado con {len(dfDepara)} registros")

dfDepara.head()

,Codigo Entidad,Razón Social,Entidad,Tipo Entidad
0,1002,BANCO NACIONAL DE FOMENTO,BNF,Participación Estatal
1,1003,BANCO DE LA NACION ARGENTINA,BNA,Suc. Direct. Extranj
2,1004,BANCO GNB PARAGUAY S.A.,GNB,P. Extranj. Mayorit.
3,1005,BANCO DO BRASIL S.A.,Do Brasil,Suc. Direct. Extranj
4,1006,CITIBANK N.A.,Citibank,Suc. Direct. Extranj


In [9]:
dfDepara.to_sql(name='DeParabanco', con=conexion_str, if_exists='replace',index=False)

21